In [19]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import xgboost as xgb

# -----------------------------
# 1. Загрузка данных
# -----------------------------
fires = pd.read_csv('C:\\Users\\Dobby\\Downloads\\data\\fires_agr.csv', header=None)
fires.columns = ['Марка', 'Склад', 'Дата возгорания', 'Формирование штабеля', 'Штабель']

temp = pd.read_csv('C:\\Users\\Dobby\\Downloads\\data\\temprerature_agr.csv')
supplies = pd.read_csv('C:\\Users\\Dobby\\Downloads\\data\\supplies_agr.csv', header=None)
supplies.columns = ['Дата поступления', 'Марка', 'Склад', 'Дата отправления', 'Штабель']

weather = pd.read_csv('C:\\Users\\Dobby\\Downloads\\data\\weather_agr.csv', header=None, skiprows=1)
weather.columns = ['datetime', 'temp_air', 'pressure', 'humidity', 'precip', 'wind']

# -----------------------------
# 2. Преобразование дат с обработкой ошибок
# -----------------------------
fires['date_fire'] = pd.to_datetime(fires['Дата возгорания'], errors='coerce')
fires = fires.dropna(subset=['date_fire'])

temp['date'] = pd.to_datetime(temp['Дата акта'], errors='coerce')
temp = temp.dropna(subset=['date'])

supplies['stack_start_date'] = pd.to_datetime(supplies['Дата поступления'], errors='coerce')
supplies = supplies.dropna(subset=['stack_start_date'])

# -----------------------------
# 3. Создание stack_id и нормализация типов
# -----------------------------
temp['stack_id'] = temp['Склад'].astype(str) + '_' + temp['Штабель'].astype(str)
fires['stack_id'] = fires['Склад'].astype(str) + '_' + fires['Штабель'].astype(str)

# Приведём даты к .date()
fires['date_fire'] = fires['date_fire'].dt.date
temp['date'] = temp['date'].dt.date

# -----------------------------
# 4. Метка: возгорание ТОГО ЖЕ штабеля в окне ±2 дня
# -----------------------------
fire_events = set(zip(fires['stack_id'], fires['date_fire']))

def has_fire_in_window(row):
    for delta in range(-2, 3):  # -2, -1, 0, +1, +2
        candidate = row['date'] + timedelta(days=delta)
        if (row['stack_id'], candidate) in fire_events:
            return 1
    return 0

temp['target'] = temp.apply(has_fire_in_window, axis=1)

# -----------------------------
# 5. Возраст штабеля (в днях)
# -----------------------------
supply_start = supplies.groupby(['Склад', 'Штабель'])['stack_start_date'].min().reset_index()
supply_start['stack_id'] = supply_start['Склад'].astype(str) + '_' + supply_start['Штабель'].astype(str)
stack_age_map = dict(zip(supply_start['stack_id'], supply_start['stack_start_date']))

temp['stack_start'] = temp['stack_id'].map(stack_age_map)
temp['stack_start'] = pd.to_datetime(temp['stack_start']).dt.date
temp['age_days'] = (pd.to_datetime(temp['date']) - pd.to_datetime(temp['stack_start'])).dt.days
temp['age_days'] = temp['age_days'].clip(lower=0).fillna(0)

# -----------------------------
# 6. Погода: агрегация до дня
# -----------------------------
weather['date'] = pd.to_datetime(weather['datetime']).dt.date
weather_daily = weather.groupby('date').agg({
    'temp_air': 'max',
    'humidity': 'mean',
    'precip': 'sum'
}).reset_index()

# Присоединяем погоду по дате
temp = temp.merge(weather_daily[['date', 'temp_air', 'humidity', 'precip']], on='date', how='left')

# -----------------------------
# 7. Динамика температуры: рост за 3 дня
# -----------------------------
temp = temp.sort_values(['stack_id', 'date']).reset_index(drop=True)
temp['temp_lag3'] = temp.groupby('stack_id')['Максимальная температура'].shift(3)
temp['temp_delta_3d'] = temp['Максимальная температура'] - temp['temp_lag3']
temp['temp_delta_3d'] = temp['temp_delta_3d'].fillna(0)

# -----------------------------
# 8. Подготовка финального датасета
# -----------------------------
temp = temp.dropna(subset=['Максимальная температура'])

# Заполнение пропусков в погоде
for col in ['temp_air', 'humidity', 'precip']:
    temp[col] = temp[col].fillna(method='ffill').fillna(method='bfill')

feature_cols = [
    'Максимальная температура',
    'age_days',
    'temp_air',
    'humidity',
    'precip',
    'temp_delta_3d'
]

X = temp[feature_cols]
y = temp['target']

# Удаляем строки с оставшимися NaN
mask = X.notnull().all(axis=1)
X = X[mask]
y = y[mask]

# -----------------------------
# 9. Обучение модели
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

scale_pos_weight = sum(y == 0) / sum(y == 1)

model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss'
)
model.fit(X_train, y_train)

# -----------------------------
# 10. Оценка
# -----------------------------
y_pred = model.predict(X_test)
print("F1-score (по классу 'возгорание'):", f1_score(y_test, y_pred, pos_label=1))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# -----------------------------
# 11. Важность признаков
# -----------------------------
importance = model.feature_importances_
feat_imp = pd.DataFrame({
    'feature': feature_cols,
    'importance': importance
}).sort_values('importance', ascending=False)

print("\nВажность признаков:")
print(feat_imp)

# -----------------------------
# 12. Предсказания: вероятности и бинарный результат
# -----------------------------
# Предсказываем для ВСЕХ строк из финального датасета
X_final = temp[mask][feature_cols]
y_proba = model.predict_proba(X_final)[:, 1]  # вероятность класса "возгорание"
y_pred_final = (y_proba >= 0.5).astype(int)

# Формируем результат
result_df = temp[mask][['Склад', 'Штабель', 'Марка', 'Дата акта', 'Максимальная температура']].copy()
result_df['Вероятность_возгорания'] = y_proba
result_df['Предсказание'] = y_pred_final

# Выводим первые 10 предсказаний на экран
print("\n🔍 Первые 10 предсказаний:")
print(result_df.head(100).to_string(index=False))

# -----------------------------
# 13. Сохранение в JSON для API
# -----------------------------
# Преобразуем дату в строку
result_df['Дата'] = pd.to_datetime(result_df['Дата акта']).dt.strftime('%Y-%m-%d')

# Выбираем итоговые колонки
output_json = result_df[[
    'Склад', 'Штабель', 'Марка', 'Дата',
    'Максимальная температура', 'Вероятность_возгорания', 'Предсказание'
]].to_dict(orient='records')

# Сохраняем в JSON
import json
with open('C:\\Users\\Dobby\\Downloads\\data\\coal_fire_predictions.json', 'w', encoding='utf-8') as f:
    json.dump(output_json, f, ensure_ascii=False, indent=2)

print("\n Предсказания сохранены в 'coal_fire_predictions.json'")
print("Пример записи для API:")
print(json.dumps(output_json[0] if output_json else {}, ensure_ascii=False, indent=2))

C:\Users\Dobby\AppData\Local\Temp\ipykernel_10500\1123047387.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fires['date_fire'] = pd.to_datetime(fires['Дата возгорания'], errors='coerce')
C:\Users\Dobby\AppData\Local\Temp\ipykernel_10500\1123047387.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  supplies['stack_start_date'] = pd.to_datetime(supplies['Дата поступления'], errors='coerce')
C:\Users\Dobby\AppData\Local\Temp\ipykernel_10500\1123047387.py:97: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp[col] = temp[col].fillna(method='ffill').fillna(method='bfill')


✅ F1-score (по классу 'возгорание'): 0.8373333333333334

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95       637
           1       0.83      0.85      0.84       185

    accuracy                           0.93       822
   macro avg       0.89      0.90      0.89       822
weighted avg       0.93      0.93      0.93       822


🔥 Важность признаков:
                    feature  importance
0  Максимальная температура    0.389206
1                  age_days    0.174298
5             temp_delta_3d    0.172021
2                  temp_air    0.114033
3                  humidity    0.084348
4                    precip    0.066094

🔍 Первые 10 предсказаний:
 Склад  Штабель     Марка  Дата акта  Максимальная температура  Вероятность_возгорания  Предсказание
     3        0        A1 2020-05-29                      26.3                0.000424             0
     3       10        A1 2020-01-19                      